### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - department)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)

In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import money_precision_at_k
from src.utils import prefilter_items, postfilter_items, get_recommendations
from src.recommenders import MainRecommender


#### Используется одноуровневая модель на базе ALS. Загружаем данные.

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')
data_test_public = pd.read_csv('../raw_data/retail_test1.csv')

item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


#### Предфильтрация данных

In [4]:
n_items_before = data_train['item_id'].nunique()
data_train = prefilter_items(data_train, take_n_popular=6500, item_features=item_features)
n_items_after = data_train['item_id'].nunique()

print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 86865 to 6501


#### Рассчитываем цену каждого товара в покупке и создаем словарь средних цен.

In [5]:
data_train['price'] = data_train['sales_value'] / (np.maximum(data['quantity'], 1))
items_price = data_train.groupby('item_id')['price'].mean().reset_index()
item_features = item_features.merge(items_price, on='item_id')
items_price = dict(zip(items_price.item_id, items_price.price))

#### Создаем модель и делаем первичные предсказания для теста, с запасом для постфильтрации

In [6]:
recommender = MainRecommender(data_train)

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [8]:
users_train = data_train['user_id'].unique().tolist()

result = get_recommendations(result, users_train, recommender.get_als_recommendations,
                             recommender.overall_top_purchases, N=50)

result.head()

,user_id,actual,pre_rec
1844,2259,"[846823, 850925, 893651, 907014, 923746, 96131...","[1029743, 1082185, 1106523, 951590, 1127831, 9..."
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1127831, 901062, 1027569, 1082185, 944836, 10..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[951590, 5568378, 1044078, 870547, 1106523, 10..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[854852, 1127831, 878996, 962568, 845208, 1024..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 1127831, 1029743, 961554, 916122, 87..."


#### Постфильтрация

In [9]:
# для работы функции постфильтрации нам понадобятся дорогие товары из топа всех покупок, ранжированные по полулярности

top500 = recommender.overall_top_purchases[:500]
top_costly_items = item_features[(item_features['price'] > 7) & (item_features['item_id'].isin(top500))]
top_costly_items = [item for item in top500 if item in top_costly_items.item_id.unique()]

In [10]:
result['rec_result'] = result.apply(lambda row: postfilter_items(row['user_id'], row['pre_rec'],
                                                                 item_features,
                                                                 recommender.top_purchases,
                                                                 top_costly_items,
                                                                 recommender.overall_top_purchases,
                                                                 items_price), axis=1)
result.head(2)

,user_id,actual,pre_rec,rec_result
1844,2259,"[846823, 850925, 893651, 907014, 923746, 96131...","[1029743, 1082185, 1106523, 951590, 1127831, 9...","[874972, 1029743, 1082185, 951590, 1127831]"
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1127831, 901062, 1027569, 1082185, 944836, 10...","[874972, 856942, 940947, 1127831, 1027569]"


#### Смотрим метрику

In [11]:
result.apply(lambda row: money_precision_at_k(row['rec_result'], row['actual'], items_price, k=5), axis=1).mean()

0.13904956701831825

#### Строим прогноз для всех юзеров, смотрим метрику на public_test и сохраняем итоговые рекомендации

In [12]:
all_users = pd.concat([data, data_test_public]).drop_duplicates(subset=['user_id'])['user_id'].reset_index()
all_users = all_users.drop('index', axis=1)

In [13]:
all_users = get_recommendations(all_users, users_train, recommender.get_als_recommendations,
                             recommender.overall_top_purchases, N=50)
all_users['rec_result'] = all_users.apply(lambda row: postfilter_items(row['user_id'], row['pre_rec'],
                                                                 item_features,
                                                                 recommender.top_purchases,
                                                                 top_costly_items,
                                                                 recommender.overall_top_purchases,
                                                                 items_price), axis=1)
all_users.tail(2)

,user_id,pre_rec,rec_result
2497,1581,"[1127831, 1082185, 878996, 961554, 866211, 854...","[874972, 1029743, 6463775, 1127831, 961554]"
2498,1984,"[1082185, 1127831, 866211, 1029743, 1004906, 9...","[874972, 843171, 1082185, 1127831, 866211]"


In [14]:
all_users = all_users.drop('pre_rec', axis=1)

result_test = data_test_public.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test = result_test.merge(all_users, on='user_id')
result_test.head()

,user_id,actual,rec_result
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[874972, 856942, 940947, 1127831, 1027569]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[968072, 1106523, 8090521, 981760, 878996]"
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109...","[829291, 1092026, 951590, 1012587, 1029743]"
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[874972, 1082185, 863447, 854852, 1022254]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[874972, 1122358, 1082185, 916122, 995965]"


In [15]:
result_test.apply(lambda row: money_precision_at_k(row['rec_result'], row['actual'], items_price, k=5), axis=1).mean()

0.12168622360890062

In [16]:
all_users.to_csv('recommendations.csv', index=False)